In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sb
import os
import random as rand
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.metrics import confusion_matrix

# Data engineering

In [ ]:
!ls ../input/titanic/

In [ ]:
train_data_path = '../input/titanic/train.csv'
test_data_path = '../input/titanic/test.csv'

## Test data 
Explore test data 

In [ ]:
titanic_train = pd.read_csv(train_data_path)
titanic_train.columns



In [ ]:
titanic_train.describe()

In [ ]:
titanic_train.head()

## Test data

In [ ]:
titanic_test = pd.read_csv(test_data_path)
titanic_test.columns

In [ ]:
titanic_test.describe()

In [ ]:
titanic_test.head()

# Data preparation

## Select reduce columns 

In [ ]:
titanic_train.dtypes

In [ ]:
titanic_test.dtypes

I propose to keep Pclass,Sex, Age, SibSP,Parch,Ticket, Fare,Cabin, Embarked, Survived

In [ ]:
columns_to_keep = ['PassengerId','Pclass', 'Sex',  'Age',  'SibSp', 'Parch','Ticket', 'Fare','Cabin', 'Embarked', 'Survived']
titanic_train = titanic_train.loc[:,columns_to_keep]
titanic_train.head()

In [ ]:
columns_to_keep = ['PassengerId','Pclass', 'Sex',  'Age',  'SibSp', 'Parch','Ticket', 'Fare','Cabin', 'Embarked']
titanic_test = titanic_test.loc[:,columns_to_keep]
titanic_test.head()

# Passengers ID
Drop. 
old : Transforms to float

In [ ]:
#titanic_train.drop("PassengerId", axis = 1)
#titanic_test.drop("PassengerId", axis = 1)
titanic_train["PassengerId"] = titanic_train["PassengerId"].astype(float)
titanic_test["PassengerId"]  = titanic_test["PassengerId"].astype(float)

# Number of sibling

In [ ]:
print(titanic_train["SibSp"].describe())
plt.hist(titanic_train["SibSp"])

In [ ]:
print(titanic_test["SibSp"].describe())
plt.hist(titanic_test["SibSp"])

In [ ]:
def categorise_siblings(data):
    cut_labels_9 = ['sib_0','sib_1','sib_2','sib_3', 
                    'sib_4','sib_5','sib_6','sib_7', 'sib_8']
    cut_bins = [0,1,2,3,4,5,6,7,8,9]
    data['Sib_cat'] = pd.cut(data['SibSp'], 
                             bins=cut_bins, 
                             labels=cut_labels_9)
    
    data['Sib_cat'] = data.Sib_cat.astype(str)
    data.loc[data["Sib_cat"] == 'nan', "Sib_cat"] = "Sib_Unknown"
    
    return data

In [ ]:
def transform_sibling_cat(data):
    factors = data['Sib_cat'].unique()
    gender_columns = pd.get_dummies(data['Sib_cat'])
    columns = range(0,len(factors))
    for column in columns:
        data[factors[column]] = gender_columns.loc[:, factors[column]].astype(float)
        
    return data

In [ ]:
titanic_train = categorise_siblings(titanic_train)
titanic_train = transform_sibling_cat(titanic_train)
titanic_train = titanic_train.drop("SibSp", axis = 1)
titanic_train = titanic_train.drop("Sib_cat", axis = 1)
titanic_train.dtypes

In [ ]:
titanic_train.shape

In [ ]:
titanic_test = categorise_siblings(titanic_test)
titanic_test = transform_sibling_cat(titanic_test)
titanic_test = titanic_test.drop("SibSp", axis = 1)
titanic_test = titanic_test.drop("Sib_cat", axis = 1)
titanic_test.dtypes

In [ ]:
titanic_test.shape

# Transforming age into categories
The categorise the age into 9 categories; unknown and one for each decade. The categories are then transformed in hot_coding format. 

titanic_train['Age'].unique()

In [ ]:
plt.hist(titanic_train['Age'], bins=10)
plt.show()

In [ ]:
titanic_train['Age'].describe()

In [ ]:
plt.hist(titanic_test['Age'], bins=10)
plt.show()

In [ ]:
titanic_test['Age'].describe()

In [ ]:
def transform_age_cat(data):
    factors = data['Age_cat'].unique()
    gender_columns = pd.get_dummies(data['Age_cat'])
    columns = range(0,len(factors))
    for column in columns:
        data[factors[column]] = gender_columns.loc[:, factors[column]].astype(float)
        
    return data


In [ ]:
def categorise_age(data):
    cut_labels_8 = ['age_0-9','age_10-19','age_20-29','age_30-39', 
                    'age_40-49','age_50-59','age_60-69','age_70-79']
    cut_bins = [0,10,20,30,40,50,60,70,80]
    data['Age_cat'] = pd.cut(data['Age'], 
                             bins=cut_bins, 
                             labels=cut_labels_8)
    data['Age_cat'] = data.Age_cat.astype(str)
    data.loc[data["Age"].isna(), "Age_cat"] = "Age_Unknown"
    return data

In [ ]:
titanic_train = categorise_age(titanic_train)
titanic_train = transform_age_cat(titanic_train)
titanic_train = titanic_train.drop("Age", axis = 1)
titanic_train = titanic_train.drop("Age_cat", axis = 1)
titanic_train.dtypes

In [ ]:
titanic_test = categorise_age(titanic_test)
titanic_test = transform_age_cat(titanic_test)
titanic_test = titanic_test.drop("Age", axis = 1)
titanic_test = titanic_test.drop("Age_cat", axis = 1)
titanic_test.dtypes

## Gender transformation to hot-coding 
We check the factor values are the same between both datasets. Then, we generate a hot coding of two columns; i.e., male and female. Both columns replace the Sex column.

In [ ]:
titanic_train['Sex'].unique()

In [ ]:
titanic_test['Sex'].unique()

In [ ]:
def transform_gender(data):
    factors = data['Sex'].unique()
    gender_columns = pd.get_dummies(data['Sex'])
    columns = range(0,len(factors))
    
    for column in columns:
        data[factors[column]] = gender_columns.loc[:,factors[column]].astype(float)
        
    return data
    

In [ ]:
titanic_train = transform_gender(titanic_train)
titanic_train.drop("Sex", axis = 1, inplace = True)
titanic_train.dtypes

In [ ]:
titanic_test = transform_gender(titanic_test)
titanic_test.drop("Sex", axis = 1,inplace = True)
titanic_test.head()

## Cabin and Pclass

The passenger class appears to drive whether a cabin is known. So, we propose to drop the cabin as the percentage of not known values is quite high. We apply an hot encoding the Pclass. 

In [ ]:
titanic_train['Cabin'].unique()

In [ ]:
print("percentage of cabin  nan values - training ", titanic_train['Cabin'].isna().sum()/titanic_train.shape[0])
print("percentage of cabin  nan values - test ", titanic_test['Cabin'].isna().sum()/titanic_test.shape[0])

In [ ]:
titanic_train['Pclass'].unique()

In [ ]:
titanic_test['Pclass'].unique()

In [ ]:
titanic_train.loc[titanic_train['Pclass'] == 1 ,['Pclass','Cabin']]

In [ ]:
titanic_train.loc[titanic_train['Pclass'] == 2 ,['Pclass','Cabin']]

In [ ]:
titanic_train.loc[titanic_train['Pclass'] == 3 ,['Pclass','Cabin']]

In [ ]:
xs = titanic_train.loc[titanic_train['Fare'] > 0,'Pclass']
ys = np.log10(titanic_train.loc[titanic_train['Fare'] > 0,'Fare'])
plt.scatter(xs,ys)

In [ ]:
xs = titanic_test.loc[titanic_test['Fare'] > 0,'Pclass']
ys = np.log10(titanic_test.loc[titanic_test['Fare'] > 0,'Fare'])
plt.scatter(xs,ys)

In [ ]:
plt.scatter(titanic_train["Pclass"],titanic_train["Fare"])

In [ ]:
def transform_Pclass(data):
    factors = data['Pclass'].unique()
    Pclass_columns = pd.get_dummies(data['Pclass'])
    columns = range(0,len(factors))
    
    for column in columns:
        col_name = 'Class_' + str(factors[column])
        data[col_name] = Pclass_columns.loc[:,factors[column]].astype(float)
        
    data.drop("Pclass", axis = 1)
    return data
    

In [ ]:
titanic_train = transform_Pclass(titanic_train)
titanic_train.drop("Pclass", axis = 1, inplace = True)
titanic_train.drop("Cabin", axis = 1, inplace = True)
titanic_train.head()

In [ ]:
titanic_test = transform_Pclass(titanic_test)
titanic_test.drop("Pclass", axis = 1, inplace = True)
titanic_test.drop("Cabin", axis = 1, inplace = True)
titanic_test.head()

# Tickets and Fare
We remove the tickets, as it brings no additional characteristic for the prediction.

Old version: We reduce the complexity of the Fare by using the log.
New version: The price appears to be dependent on the class, so we drop the price.

In [ ]:
titanic_train.drop("Ticket", axis = 1, inplace = True)
titanic_test.drop("Ticket", axis = 1, inplace = True)

In [ ]:
log_10_values = np.log10(titanic_train.loc[titanic_train['Fare'] > 0,'Fare'])
titanic_train.loc[titanic_train['Fare'] > 0,'Fare'] = log_10_values
titanic_train.Fare.describe()


In [ ]:
log_10_values = np.log10(titanic_test.loc[titanic_test['Fare'] > 0,'Fare'])
titanic_test.loc[titanic_test['Fare'] > 0,'Fare'] = log_10_values
titanic_test.Fare.describe()

In [ ]:
titanic_train.drop("Fare", axis = 1, inplace = True)
titanic_test.drop("Fare", axis = 1, inplace = True)

# Embarked 
Embarked has three values "S", 'C', 'Q' and some values unknown. So, we replace the unknown values with U. A hot-encoding transforms the values into columns. 

The test datasets has no unknown values; so we had a column 'U' for consistency with hot-encoding. 


In [ ]:
titanic_train.Embarked.unique()

In [ ]:
titanic_train.loc[titanic_train['Embarked'].isna(),'Embarked'] = 'U'
titanic_train.Embarked.unique()

In [ ]:
titanic_test.loc[titanic_test['Embarked'].isna(),'Embarked'] = 'U'
titanic_test.Embarked.unique()

In [ ]:
def transform_embarked(data):
    factors = data['Embarked'].unique()
    gender_columns = pd.get_dummies(data['Embarked'])
    columns = range(0,len(factors))
    
    for column in columns:
        data[factors[column]] = gender_columns.loc[:,factors[column]].astype(float)
        
    return data
    

In [ ]:
titanic_train = transform_embarked(titanic_train)
titanic_train.drop("Embarked", axis = 1, inplace = True)
titanic_test  = transform_embarked(titanic_test)
titanic_test.drop("Embarked", axis = 1, inplace = True)


In [ ]:
indices       = range(0, titanic_test.shape[0])
titanic_test['U'] = [0 for i in indices]
titanic_test['U'] = titanic_test['U'].astype(float)

# Outcome of data preparations

In [ ]:

print("training datasets : " , titanic_train.shape)
titanic_train.dtypes

In [ ]:
print("testing datasets : " , titanic_test.shape)
titanic_test.dtypes

In [ ]:
train_cols = titanic_train.columns
test_cols = titanic_test.columns

common_cols = train_cols.intersection(test_cols)
train_not_test = train_cols.difference(test_cols)
train_not_test

In [ ]:
titanic_test.columns

# Cross validation preparation
We use a stratified sampling for the training into a train and test dataset. 

In [ ]:
x_cols = ['Parch', 'Sib_Unknown', 'sib_0', 'sib_1', 'sib_2', 'sib_3', 'sib_4', 'sib_7', 'age_30-39', 'age_40-49', 'age_60-69', 'age_20-29', 'age_10-19', 'Age_Unknown', 'age_50-59',
       'age_0-9', 'age_70-79', 'male', 'female', 'Class_3', 'Class_2',
       'Class_1', 'Q', 'S', 'C', 'U']
y_col = 'Survived'
X = titanic_train.copy(deep = True)
X = X[x_cols]
X = X.apply(pd.to_numeric)
y = titanic_train[y_col].apply(pd.to_numeric)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
for train_index, test_valid_index in split.split(X, y):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test  = X.iloc[test_valid_index]
    y_test  = y.iloc[test_valid_index]

# we see our training set follows the same distribution
print(y_train.value_counts(normalize=True), '\n\n')

# we see our test set follows the same distribution
print(y_test.value_counts(normalize=True))

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train_encode=pd.get_dummies(y_train)
y_test_encode=pd.get_dummies(y_test)


In [ ]:
train_cols = X_train.columns
test_cols = titanic_test.columns

common_cols = train_cols.intersection(test_cols)
train_not_test = train_cols.difference(test_cols)
train_not_test

In [ ]:


no_columns = X_train.shape[1]
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(no_columns,)))
model.add(tf.keras.layers.Dense(50, activation="sigmoid"))
model.add(tf.keras.layers.Dense(50, activation="sigmoid"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))


version_1
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(560, activation="sigmoid"))
model.add(tf.keras.layers.Dense(1120, activation="sigmoid"))
model.add(tf.keras.layers.Dense(1120, activation="sigmoid"))
model.add(tf.keras.layers.Dense(560, activation="sigmoid"))
model.add(tf.keras.layers.Dense(560, activation="sigmoid"))
model.add(tf.keras.layers.Dense(2, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
best_recall = 0.0
learning_rates = np.arange(0.0001,0.0005,0.00001)

for rate in learning_rates:
    print("=== rates ===", rate)
    opt = tf.keras.optimizers.Adam(learning_rate = rate)
    model.compile(optimizer= opt, 
                  loss = "binary_crossentropy",
                  metrics=["accuracy"])

    history = model.fit(X_train,
                        y_train_encode,
                        validation_data=(X_test, y_test_encode),
                        epochs = 10,
                        verbose = False)
    y_pred = model.predict(X_test)
    Y_pred = np.argmax(model.predict(X_test),axis=1)
    cm = confusion_matrix(y_test, Y_pred)
    recall = cm[1][1]/(cm[0][1] + cm[1][1]) #TP/(TP+FN)
    
    if recall > best_recall:
        best_recall = recall
        best_rate   = rate 
        print("=== best recall ===" , recall)

    if best_recall == 1.0:
        break

In [ ]:

opt = tf.keras.optimizers.Adam(learning_rate = best_rate)
model.compile(optimizer= opt, 
              loss = "binary_crossentropy",
              metrics=["accuracy"])

history = model.fit(X_train,
                    y_train_encode,
                    validation_data=(X_test, y_test_encode),
                    epochs = 800,
                    verbose = False)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'ro', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
_, accuracy = model.evaluate(X_test, y_test_encode)
print('Accuracy: %.4f' % (accuracy*100))

In [ ]:
y_pred = model.predict(X_test)
Y_pred = np.argmax(model.predict(X_test),axis=1)
cm = confusion_matrix(y_test, Y_pred)
recall_model = cm[1][1]/(cm[0][1] + cm[1][1]) #TP/(TP+FN)
    

# Prediction using the test csv file

In [ ]:
X = titanic_test.copy(deep = True)
X = X[x_cols]
X = X.apply(pd.to_numeric)
X.columns



In [ ]:
y_pred = model.predict(X)


# Hyper-parametrisation of architecture random-walk

In [ ]:
import random

best_recall_overall = recall_model

attempts = range(0,100)


for attempt in attempts:
    no_columns = X_train.shape[1]
    layers     = range(2, random.randint(3,10))
    no_nodes   = random.randint(20,50)
    
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(no_columns,)))
    
    for layer in layers:
        model.add(tf.keras.layers.Dense(no_nodes, activation="sigmoid"))
    
    
    model.add(tf.keras.layers.Dense(2, activation="softmax"))
    
    print(model.summary())
    
    best_recall = 0.0
    learning_rates = np.arange(0.0001,0.0005,0.00001)
    for rate in learning_rates:
        print("=== rates ===", rate)
        opt = tf.keras.optimizers.Adam(learning_rate = rate)
        model.compile(optimizer= opt, 
                  loss = "binary_crossentropy",
                  metrics=["accuracy"])

        history = model.fit(X_train,
                        y_train_encode,
                        validation_data=(X_test, y_test_encode),
                        epochs = 10,
                        verbose = False)
        y_pred = model.predict(X_test)
        Y_pred = np.argmax(model.predict(X_test),axis=1)
        cm = confusion_matrix(y_test, Y_pred)
        recall = cm[1][1]/(cm[0][1] + cm[1][1]) #TP/(TP+FN)
    
        if recall > best_recall:
            best_recall = recall
            best_rate   = rate 
            print("=== best recall ===" , recall)

        if best_recall == 1.0:
            learning_rates = []

    opt = tf.keras.optimizers.Adam(learning_rate = best_rate)
    model.compile(optimizer= opt, 
              loss = "binary_crossentropy",
              metrics=["accuracy"])

    history = model.fit(X_train,
                    y_train_encode,
                    validation_data=(X_test, y_test_encode),
                    epochs = 800,
                    verbose = False)
    
    _, accuracy = model.evaluate(X_test, y_test_encode)
    print('Accuracy: %.4f' % (accuracy*100))
    
    y_pred = model.predict(X_test)
    Y_pred = np.argmax(model.predict(X_test),axis=1)
    cm = confusion_matrix(y_test, Y_pred)
    recall = cm[1][1]/(cm[0][1] + cm[1][1]) #TP/(TP+FN)
    
    if recall > best_recall_overall:
        best_recall_overall = recall
        best_model = tf.keras.models.clone_model(model)
        print("=== best recall overall ===" , recall)
    

In [ ]:
best_model.summary()